In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from joblib import dump, load
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import glob

import matplotlib as mpl

import matplotlib.pyplot as plt

import math as mt

import cartopy.crs as ccrs

import cartopy.feature as cfeature

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

#AO Calculation

In [2]:
# Calculate AO - input std normal latitude wighted 1000mb z
ZData = xr.open_dataset('../../../Data/CMIP6/IPSL-CM6A-LR/NDJF/weighted1000Std.nc')
#SLPData = xr.open_dataset('../eraDown/ERA5_2degree_Down/DailyMean/weighted1000.nc')

ZData.zg.values
ZData.zg.values=xr.where(np.isnan(ZData.zg.values),  0.000000000001,ZData.zg.values)
ZData.zg.values

array([[[1.0000000e-12, 1.0000000e-12, 1.0000000e-12, ...,
         1.0000000e-12, 1.0000000e-12, 1.0000000e-12],
        [1.0000000e-12, 1.0000000e-12, 1.0000000e-12, ...,
         1.0000000e-12, 1.0000000e-12, 1.0000000e-12],
        [1.0000000e-12, 1.0000000e-12, 1.0000000e-12, ...,
         1.0000000e-12, 1.0000000e-12, 1.0000000e-12],
        ...,
        [4.3787098e+01, 4.3882660e+01, 4.3970936e+01, ...,
         4.3484932e+01, 4.3587749e+01, 4.3690601e+01],
        [3.0332964e+01, 3.0326052e+01, 3.0321266e+01, ...,
         3.0328671e+01, 3.0333899e+01, 3.0332882e+01],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00]],

       [[1.0000000e-12, 1.0000000e-12, 1.0000000e-12, ...,
         1.0000000e-12, 1.0000000e-12, 1.0000000e-12],
        [1.0000000e-12, 1.0000000e-12, 1.0000000e-12, ...,
         1.0000000e-12, 1.0000000e-12, 1.0000000e-12],
        [1.0000000e-12, 1.0000000e-12, 1.0000000e-12, ...,
         1.000

In [3]:


t1=ZData.zg.stack(z=("lat", "lon"))
# fit scaler on training data
zg_PC =PCA().fit_transform(t1)


In [4]:
print(zg_PC[:,0].shape)
AO_ds = xr.Dataset({'AO': (('time'), zg_PC[:,0])}, coords={'time': ZData.time})


norm = StandardScaler().fit(zg_PC[:,0].reshape(-1, 1))
    # transform training data
aoX = norm.transform(zg_PC[:,0].reshape(-1, 1))
aoX.shape

AO_ds = xr.Dataset({'AO': (('time'), aoX[:,0]*-1.0)}, coords={'time': ZData.time})


(4810,)


In [5]:
AO_ds.to_netcdf('IPSL-CM6A-LR-AOindex-NDJF-Daily-1980-2014.nc')

In [6]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.max(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(2.3589196, dtype=float32)
Coordinates:
    time     datetime64[ns] 1995-12-02T12:00:00

In [7]:
#AO_ds.AO.where(AO_ds.AO>2.0, drop=True).squeeze()

In [8]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.min(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(-3.8414636, dtype=float32)
Coordinates:
    time     datetime64[ns] 1999-02-03T12:00:00

In [9]:
#AO_ds.AO.where(AO_ds.AO<-2, drop=True).squeeze()

#EU Calculation

In [10]:
zLevData = xr.open_dataset('../../../Data/CMIP6/IPSL-CM6A-LR/NDJF/zg_NDJFday_IPSL-CM6A-LR_historical_r1i1p1f1_gr_19750101-20141231.nc')

In [11]:
zLevData.plev

<xarray.DataArray 'plev' (plev: 8)>
array([100000.,  85000.,  70000.,  50000.,  25000.,  10000.,   5000.,   1000.],
      dtype=float32)
Coordinates:
  * plev     (plev) float32 1e+05 8.5e+04 7e+04 5e+04 2.5e+04 1e+04 5e+03 1e+03
Attributes:
    standard_name:  air_pressure
    long_name:      pressure
    units:          Pa
    positive:       down
    axis:           Z
    name:           plev

In [12]:
z500=zLevData.zg.sel(plev=50000.0,method='nearest')
z500

<xarray.DataArray 'zg' (time: 4810, lat: 143, lon: 144)>
[99047520 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1975-01-01T12:00:00 ... 2014-12-31T12:00:00
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat      (lat) float32 -90.0 -88.73 -87.46 -86.2 ... 86.2 87.46 88.73 90.0
    plev     float32 5e+04
Attributes:
    standard_name:       geopotential_height
    long_name:           Geopotential Height
    units:               m
    online_operation:    average
    cell_methods:        time: mean
    interval_operation:  900 s
    interval_write:      1 d
    description:         Geopotential Height
    history:             none
    cell_measures:       area: areacella

In [13]:
z145E40N=z500.sel(lat=40,lon=145,method='nearest')

In [14]:
#z145E40N

In [15]:
z20E55N=z500.sel(lat=55,lon=20,method='nearest')

In [16]:
#z20E55N

In [17]:
z75E55N=z500.sel(lat=55,lon=75,method='nearest')

In [18]:
#z75E55N

In [19]:
#Calculate Anomaly
z145E40N_Anom = z145E40N.groupby("time.dayofyear") - z145E40N.groupby("time.dayofyear").mean("time")
z75E55N_Anom  = z75E55N.groupby("time.dayofyear")  - z75E55N.groupby("time.dayofyear").mean("time")
z20E55N_Anom  = z20E55N.groupby("time.dayofyear")  - z20E55N.groupby("time.dayofyear").mean("time")

In [20]:
# calculate std normal anomalies
z145E40N_AnomStd = z145E40N_Anom.groupby("time.dayofyear")  / z145E40N.groupby("time.dayofyear").std("time")
z75E55N_AnomStd  = z75E55N_Anom.groupby("time.dayofyear")   / z75E55N.groupby("time.dayofyear").std("time")
z20E55N_AnomStd  = z20E55N_Anom.groupby("time.dayofyear")   / z20E55N.groupby("time.dayofyear").std("time")

In [21]:
EUVal = -(1.0/4.0)*z20E55N_AnomStd + (1.0/2.0)*z75E55N_AnomStd - (1.0/4.0)*z145E40N_AnomStd

In [22]:
EUVal

<xarray.DataArray 'zg' (time: 4810)>
array([0.42460358, 0.2347138 , 0.22528634, ..., 0.13304345, 0.53359026,
       0.94087796])
Coordinates:
  * time       (time) datetime64[ns] 1975-01-01T12:00:00 ... 2014-12-31T12:00:00
    plev       float32 5e+04
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365
    lon        float32 145.0

In [23]:
EU_ds = xr.Dataset({'EU': (('time'), EUVal)}, coords={'time': zLevData.time})

In [24]:
EU_ds.to_netcdf('IPSL-CM6A-LR-EUindex-NDJF-Daily-1980-2014.nc')